# Homework 3 - Outputs file

In [2]:
import functions_hw3 as f

### List of paths

In [3]:
path_list_anime = "list_anime.txt"
url_of_single_page = "https://myanimelist.net/topanime.php?limit="
path_page_general = "Anime/Page"
path_documents_file = "documents.tsv"
path_vocabulary_file = "vocabulary.json"
path_index_file = "Anime/index.json"
path_documents_ex_3 = "Anime/documents_ex_3.tsv"
path_vocabulary_title_file = "/Anime/vocabulary_title.json"
path_index_title_file ="Anime/index_title.json"
path_inverted_index_file="Anime/inverted_index.json"
path_inverted_index_title_file = "Anime/inverted_index_title.json"

## 1. Data Collection

### 1.1. Get the list of animes

#### Write a file with all anime url

In [ ]:
f.write_list_anime(path_list_anime, url_of_single_page)

### 1.2. Crawl animes

#### Download all html

In [ ]:
f.download_html(path_list_anime, path_page_general)

### 1.3 Parse downloaded pages

#### This function will create 3 different files: 
- one .tsv file for each anime
- one .tsv file with 3 most important information for all animes: Title, Description, Url
- one .tsv file with 4 most important information for all animes: Title, Description, Url, AnimeScore

In [ ]:
f.create_tsvs(path_list_anime, path_page_general, path_documents_file, path_documents_ex_3)

## 2. Search Engine

#### Create pandas dataframe with important information from previous written files

In [ ]:
f.df_from_3_information_file(path_documents_file)

In [ ]:
f.df_from_4_information_file(path_documents_ex_3)

#### Text cleaning:

In order we will do the following operations:

- Remove "[Written by MAL Rewrite]" at the end of each description

- Remove Contractions (for example "won't" become "will not" and "don't" become "do not")

- Make all characters lower case (for example "Hello" become "hello")

- Remove dashes

- Remove ordinal numbers (for example 1st, 2nd, ...)

- Remove stopwords (adding to stopwords frequent words that appear in descriptions, for example "character", "end") 

- Remove punctuation

- Stemming

All the functions are implemented in functions_hw3.py file

#### Function that apply text cleaning on a precise column of a pandas dataframe and returns a pandas dataframe with only this cleaned column. 
The parameter "original_df" is one of the two obtained with previous functions "df_from_3_information_file" and "df_from_4_information_file".

The parameter "column_type" accept a column with some text or string inside.

In [ ]:
f.clean_df(original_df, column_type)

### 2.1. Conjunctive query

### 2.1.1) Create your index!

#### Create vocabulary and first inverted index (named index for simplicity) and save them as .json files

In [ ]:
f.write_vocabulary(path_vocabulary_file, df_descriptions)

In [ ]:
f.write_index(vocabulary, path_index_file, df_descriptions):

#### Functions to read vocabulary and index from files

In [ ]:
f.read_vocabulary(path_vocabulary_file)

In [ ]:
f.read_index(path_index_file)

### 2.1.2) Execute the query

#### Execute the conjunctive query

This function takes in input a string as "query", the vocabulary and the associated index for anime descriptions.

It returns a pandas.Dataframe object with all the anime that contains all the words in the query

In [3]:
query = "robot"
vocabulary = f.read_vocabulary(path_vocabulary_file)
index = f.read_index(path_index_file)

In [16]:
f.execute_query(query, vocabulary, index, path_documents_file)

,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n
45,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,https://myanimelist.net/anime/2921/Ashita_no_J...
90,Neon Genesis Evangelion: The End of Evangelion,Shinji Ikari is left emotionally comatose afte...,https://myanimelist.net/anime/38474/Yuru_Camp△...
124,Nichijou,Nichijou primarily focuses on the daily antics...,https://myanimelist.net/anime/31181/Owarimonog...
62,Tengen Toppa Gurren Lagann,Simon and Kamina were born and raised in a dee...,https://myanimelist.net/anime/34591/Natsume_Yu...


### 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

#### Create second inverted index (named inverted_index for simplicity) and save it as .json file

This function take similar inputs as previuos write files functions

In [ ]:
f.write_inverted_index (original_df, vocabulary, index, column_type, path_inverted_index_file)

#### Function to read inverted index from file

In [ ]:
f.read_inverted_index(path_inverted_index_file)

#### Function that return the list of indexes of conjunctive query

In [ ]:
f.docs_contains_query(query, vocabulary, index, path_documents_file)

#### Given a query, we calculate some vectors as numpy array for optimization (based on "animeDescription" column in the dataframe):
- query vector of lenght equals to the lenght of vocabulary, with ones if the word is present and zeros otherwise **function get_query_vector(query, vocabulary)**
- doc vector for each documents in the total original cleaned dataframe **get_doc_vector(n_row_doc, df_cleaned, vocabulary, inverted_index, column_type)**
- score as cosine similarity between query vector and the selected document vector in loop **calc_score(q_vector, doc_vector)**

### 2.2.2) Execute the query

#### Then it's possible to execute the query

It will return a list of documents, ranked by their Cosine Similarity with respect to the query entered in input.

In [4]:
query = "robot"
k = 5
column_type = "animeDescription"

In [5]:
original_df = f.df_from_3_information_file(path_documents_file)
df_cleaned = f.clean_df(original_df, column_type)

In [6]:
inverted_index = f.read_inverted_index(path_inverted_index_file)

In [9]:
f.execute_similarity_query(query, k, df_cleaned, vocabulary, index, inverted_index, original_df, path_documents_file, column_type)

,animeTitle,animeDescription,Url,Similarity animeDescription
124,Nichijou,Nichijou primarily focuses on the daily antics...,https://myanimelist.net/anime/31181/Owarimonog...,0.080656
45,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,https://myanimelist.net/anime/2921/Ashita_no_J...,0.051465
90,Neon Genesis Evangelion: The End of Evangelion,Shinji Ikari is left emotionally comatose afte...,https://myanimelist.net/anime/38474/Yuru_Camp△...,0.050071
62,Tengen Toppa Gurren Lagann,Simon and Kamina were born and raised in a dee...,https://myanimelist.net/anime/34591/Natsume_Yu...,0.050121
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n,0.049967


## 3. Define a new score!

#### Explanation of the new score

We decided to build the new score based on a weighted average between cosine score calculated on anime descriptions, cosine distance on anime title and considering the field anime Score (normalized)

We build a function with multiple chance to choose the score (the user can activate a sort of filter, deciding which fields will be used in the final score calculation between these two new).

In the weighted average, if the cosine distance of anime title is considered, it will have a weight equals to **2**. If the anime score is considered it will have weight equals one, but its value is rescaled divided by its max value (**10**). In both cases cosine score calculated on anime descriptions will have weight equals to **1**

By default, the filters are set to false, so the score is equal to the previous excercise.

#### Before execution of the query we have to write (and read after of course) vocabulary, index and inverted index for "animeTitle" field with the follwing methods:

In [ ]:
f.write_vocabulary_title(path_vocabulary_title_file, df_title)

In [ ]:
f.write_index_title(vocabulary_title, path_index_title_file, df_title)

Passing as column_type : "animeTitle"

In [ ]:
f.write_inverted_index (original_df, vocabulary, index, column_type, path_inverted_index_title_file)

#### Execute the query and show results as previous excercises in a pandas dataframe structure, with the new column "FinalScore" which will show up the total final score

In [31]:
query = "robot"
k = 2
prior_title = True
prior_a_score = False

In [32]:
original_df = f.df_from_4_information_file(path_documents_ex_3)

In [33]:
vocabulary = f.read_vocabulary(path_vocabulary_file)
index = f.read_index(path_index_file)
inverted_index = f.read_inverted_index(path_inverted_index_file)

In [34]:
vocabulary_title = f.read_vocabulary(path_vocabulary_title_file)
index_title = f.read_index(path_index_title_file)
inverted_index_title = f.read_inverted_index(path_inverted_index_title_file)

In [35]:
f.execute_new_score_query(query, k, vocabulary, index, inverted_index, vocabulary_title, index_title, inverted_index_title, 
                            path_documents_file, path_documents_ex_3, prior_title = True, prior_a_score = False)

,animeTitle,animeDescription,Url,animeScore,Similarity animeDescription,Similarity animeTitle,FinalScore
index,,,,,,,
124,Nichijou,Nichijou primarily focuses on the daily antics...,https://myanimelist.net/anime/31181/Owarimonog...,8.46,0.080656,0,0.026885
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n,9.16,0.049967,0,0.016656


## 5. Algorithmic question

#### * Loop for all elements in requests
* Maintain two sums include and exclude
* include = Max sum including the previous element
* exclude = Max sum excluding the previous element
 
 
* for i in requests:
    * include = exclude + requests[i]
    * exclude = max(include,exclude)


* for selecting optimal values we will use 

* if sum(list) == exclude
* list.append(requests[i])


* during adding value we will have 3 type of situation 
1. sum(list) == exclude
2. sum(list-last element) == exclude
3. sum(list-last 2 element) == exclude


* initially

* requests = [30, 40, 25, 50, 30, 20]


* include = 30                                
* exclude = 0                                 
* list1 = [30]
* list2 = [0]

* include = exclude + requests[i]
* exclude = max(include,exclude)


* for i = 1 (element is 40)
* include = (exclude + requests[i]) = 40      
* exclude = max(30, 0) = 30                   
* list1 = [30]
* list2 = [40]


* for i = 2 (element is 25)                   
* include = (exclude + requests[i]) = 55      
* exclude = max(40, 30) = 40                  
* list1 = [30,25]
* list2 = [40]


* for i = 3 (element is 50)
* include = (exclude + requests[i]) = 90      
* exclude = max(55, 40) = 55                  
* list1 = [30,25]
* list2 = [40,50]


* for i = 4 (element is 30)                   
* include = (exclude + requests[i]) = 85      
* exclude = max(90, 55) = 90                  
* list1 = [30,25,30]
* list2 = [40,50]


* for i = 5 (element is 20)
* include = (exclude + requests[i]) = 110     
* exclude = max(85, 90) = 90                  
* list1 = [30,25,30]
* list2 = [40,50,20]


* 20 is the last element, So answer is max(110,90) = 110




In [165]:
# Function to return max sum such that no two elements are consecutive 
def find_max_time(requests):
    #creating include and exclude
    include = 0 
    exclude = 0
    #creating 2 lists for adding values
    list1 = []
    list2 = []
    #creating 2 lists for adding i+2 index value when we test i+3 index
    list1_1 = []
    list2_1 = []
    #we have to know the len of request in order to pick i+2 or i+3 in last element
    requests_len = len(requests)
    

    for i, j in zip(requests, range(requests_len)):
        # Current max excluding i
        new_exclude = exclude if exclude > include else include

        # Storing value as a list
        # before we apply "include = exclude + requests[i]" we find list that sum  is equal to exclude
        if sum(list1) == exclude:
            list1.append(i)
        elif sum(list2) == exclude:
            list2.append(i)

        # if none of them is equal to exclude we check it without last value of list    
        elif sum(list1[:-1]) == exclude:
            list1_1.append(list1[-1]) # adding last value to list1_1
            list1 = list1[:-1] # removing last value 
            list1.append(i) # adding current i
            if len(list1_1)>1: # checking stored value in list 1_1 
                list1_1 = list1_1[1:] #remove firs elemet it always store 1 element
            #Last element control
            if j == requests_len-1: # if i is last element
                if sum(list1[:-1] + list1_1) > sum(list1): #compare last element and previous element which is we stored in list1_1
                    list1 = list1[:-1] #  change last element
                    list1 = list1 + list1_1[:1]
                    
        # applying same steps for list2 and list2_1
        elif sum(list2[:-1]) == exclude:
            list2_1.append(list2[-1]) # adding last value to list2_1
            list2 = list2[:-1] # removing last value 
            list2.append(i) # adding current i
            if len(list2_1)>1: # checking stored value in list 2_1 
                list2_1 = list2_1[1:] #remove firs elemet it always store 1 element
            #Last element control
            if j == requests_len-1: # if i is last element
                if sum(list2[:-1] + list2_1) > sum(list2): #compare last element and previous element which is we stored in list1_1
                    list2 = list2[:-1] #  change last element
                    list2 = list2 + list2_1[:1]                

        # if none of previous conditions is true we replace last value in list with value is stored in list1_1            
        elif sum(list1[:-1]+list1_1) == exclude:
            list1_1.append(list1[-1]) #add last value to list1_1
            list1 = list1[:-1]
            list1 = list1 + list1_1[:1] #replace last value of list1 with first value of list1_1
            list1.append(i)
            if len(list1_1)>1:
                list1_1 = list1_1[1:] #remove 1st element from list1_1 whish is added to  list1
                
            #Last element control
            if j == requests_len-1: # check if i is last element
                if sum(list1[:-2] + list1_1) > sum(list1): # compare list with raplacing last value
                    list1 = list1[:-2]
                    list1 = list1 + list1_1 #adding back previous value
                    
        # for list2
        else:
            list2_1.append(list2[-1])#add last value to list2_1
            list2 = list2[:-1]
            list2 = list2 + list2_1[:1]#replace last value of list2 with first value of list2_1
            list2.append(i)
            if len(list2_1)>1:
                list2_1 = list2_1[1:] #remove 1st element from list2_1 whish is added to  list2
                
            #Last element control
            if j == requests_len-1:# check if i is last element
                if sum(list2[:-2] + list2_1) > sum(list2):# compare list with raplacing last value
                    list2 = list2[:-2]
                    list2 = list2 + list2_1 #adding back previous value

        # Current max including i
        include = exclude + i 
        exclude = new_exclude
    
    # return max of include, exclude and list1, list2
    return (exclude if exclude > include else include, list1 if sum(list1) > sum(list2) else list2)

In [166]:
def request():  
    # creating an empty list
    requests = []

    # number of elements as input
    n = int(input("Enter number of elements : "))

    # iterating till the range
    for i in range(0, n):
        element = int(input())

        requests.append(element) # adding the element

    print(requests)
    print ('Maximum duration: ',find_max_sum(requests))

In [167]:
request()

Enter number of elements : 7
30
40
25
50
30
20
70
[30, 40, 25, 50, 30, 20, 70]
Maximum duration:  (160, [40, 50, 70])


In [168]:
request()

Enter number of elements : 7
30
40
25
50
30
20
10
[30, 40, 25, 50, 30, 20, 10]
Maximum duration:  (110, [40, 50, 20])


In [169]:
request()

Enter number of elements : 8
30
40
25
50
30
20
10
40
[30, 40, 25, 50, 30, 20, 10, 40]
Maximum duration:  (150, [40, 50, 20, 40])


In [170]:
request()

Enter number of elements : 10
30
20
50
40
30
60
70
20
30
50
[30, 20, 50, 40, 30, 60, 70, 20, 30, 50]
Maximum duration:  (230, [30, 50, 30, 70, 50])


In [171]:
request()

Enter number of elements : 3
20
30
5
[20, 30, 5]
Maximum duration:  (30, [30])


In [172]:
request()

Enter number of elements : 5
20
30
20
50
60
[20, 30, 20, 50, 60]
Maximum duration:  (100, [20, 20, 60])


In [173]:
request()

Enter number of elements : 5
30
20
20
50
20
[30, 20, 20, 50, 20]
Maximum duration:  (80, [30, 20, 20])
